In [1]:
import json
import datetime
import time
import requests
import nbimporter
import random
import Proxy_Gatherer as pg
import Init_database as indb
import Insert_py
import Get_py
import psycopg2
from multiprocessing import Process

Importing Jupyter notebook from Proxy_Gatherer.ipynb
Importing Jupyter notebook from Init_database.ipynb
Importing Jupyter notebook from Insert_py.ipynb
Importing Jupyter notebook from Get_py.ipynb


In [2]:
# #--------THIS ONLY RUNS ONE TIME TO CREATE A DATABASE FRAMEWORK--------#
# import Create_Teld_Database as ctd
# db_init = ctd.create_teld_database()

In [3]:
#--------INITIALIZING CONNECTION TO DATABASE--------#
db_name="db_teld"
user_name="postgres"
password = "postgres"
host="127.0.0.1"
port="5432"
conn = indb.init_db(db_name, user_name, password, host, port)

In [4]:
##------- Specify request header ------- ##
hdr = {
    'accept': "*/*",
    'user-agent': "Teld/3.4.0 (iPhone; iOS 10.3.3; Scale/2.00)",
    'cookie': "TELDAppID=",
    'content-length': "111",
    'connection': "keep-alive",
    'accept-encoding': "gzip, deflate",
    'device': "app_version=3.4.0&os_version=10.3.3&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=",
    'host': "basesg.teld.cn",
    'content-type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache",
    'postman-token': "4dd0a429-35ad-1863-c8f5-18e0f7805f60"
    }

In [5]:
#------- Gathering proxies for visiting teld.cn -------##
hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',
       'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
gatherproxy = pg.ProxyGatherer(hdr)
proxies_xici = gatherproxy.getproxies_xici()
proxies_elite = gatherproxy.getproxies_elite()
# t = time.time()
proxiesXici_pool = gatherproxy.proxies_cleanup(proxies_xici) # Could also specify timeout limit you prefer, 
                                                                #format--(connect time, read time)
proxiesElite_pool = gatherproxy.proxies_cleanup(proxies_elite)
# print(time.time()-t)  ## usually this checking process takes a minute or so and usable proxies are around 80+.

Got rid of 39 unusable proxies.
Got rid of 33 unusable proxies.


In [8]:
proxiesElite_pool[0]['http']

'http://185.174.63.188:3128'

In [7]:
# F = open('Error_log.txt','w') 

In [9]:
def pillState_scrap(conn, hdr, pillcode, time_out, error_log=None, proxies_pool=None): 
    """
        hdr - dictionary, detailed spec for headers
        pillCode - string, each charging pile has its own pile code
        error_log - specify file to write errors
        proxies_pool - contains all the proxies address

    """
    F = error_log
    #If encounter error, try one more request with a different proxy
    try:
        pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
    except ValueError as e:
        if F is not None:
            F.write(pile[0])
            F.write(str(e))
            F.write('\n')
        try:
            pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
        except ValueError as e:
            if F is not None:
                F.write(pile[0])
                F.write(str(e))
                F.write('\n')
            pill_detailedInfo = None
        except (requests.exceptions.ChunkedEncodingError) as e:
            if F is not None:
                F.write(pile[0])
                F.write(str(e))
                F.write('\n')
            pill_detailedInfo = None
    except (requests.exceptions.ChunkedEncodingError) as e:
        if F is not None:
            F.write(pile[0])
            F.write(str(e))
            F.write('\n')
        try:
            pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
        except ValueError as e:
            if F is not None:
                F.write(pile[0])
                F.write(str(e))
                F.write('\n')
            pill_detailedInfo = None
        except (requests.exceptions.ChunkedEncodingError) as e:
            if F is not None:
                F.write(pile[0])
                F.write(str(e))
                F.write('\n')
            pill_detailedInfo = None
    if pill_detailedInfo is not None:
        pillCode = str(pill_detailedInfo['pillCode'])
        stateCode = str(pill_detailedInfo['stateCode'])
        stateCodeV3 = str(pill_detailedInfo['stateCodeV3'])
        if pill_detailedInfo['stateName'] is not None:
            stateName = str(pill_detailedInfo['stateName'].encode('utf-8'))
        else:
            stateName = ''
        if pill_detailedInfo['stateNameV3'] is not None:
            stateNameV3 = str(pill_detailedInfo['stateNameV3'].encode('utf-8'))
        else:
            stateNameV3 = ''
#         insertTime = time.time()
#         conn = Insert_py.insert_state_info(conn, pillCode, stateCode, stateName, stateCodeV3, stateNameV3)
#         insertTime = time.time() - insertTime
#         print(insertTime)

In [7]:
# ##------- Every once a while runs it update station and pile fixed info -------##
# #--------------------------------------------#
# #------- Gather all stations  -------# ****NEED TO BE FIXED 09.17.2017
# station_info = Get_py.get_allStations(hdr)
# print(len(station_info))
# for station in station_info:
#     stationId = str(station['id'])  
# #     print(stationId)
#     #------- Gather individual stations details -------#
#     station_detailedInfo = Get_py.get_oneStation_APP(hdr, stationId)
#     conn = Insert_py.insert_singStation_details(conn, station_detailedInfo)
#     #------- Gather individual pile code -------#
#     piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)
#     conn = Insert_py.insert_pilecode(conn, stationId, piles_info)
# #------- Gather inidividual pile details -------#
# #------- Request pilecode from db for states -------#
try:
    cur = conn.cursor()
    cur.execute("select pilecode from tbl_APP_pilecode;")
    pillcode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(pillcode_list))

# # ------- Requests stationID from db for user charging activities -------#
try:
    cur = conn.cursor()
    cur.execute("select stationid from tbl_app_eachstations;")
    station_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(station_list))

# reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15)
# i = 0
# for pile in pilecode_list:
#     #If encounter error, try one more request with a different proxy
#     try:
#         pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#     except ValueError as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     except (requests.exceptions.ChunkedEncodingError) as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     if pile_detailedInfo is not None:
#         conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)
#     else:
#         print("Running None Func.")
#         conn = Insert_py.insert_singPile_none(conn, pile_detailedInfo, pile[0])
#     i += 1
#     if datetime.datetime.now() >= reportTime:
#         reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15) 
#         print("Successfully run {} piles".format(i))
# #211 Running None Func.
# #71585 piles.

71585
8866


In [27]:
stationId = "d4035b02-5f27-4ae9-b0b1-dd10010f5b7e"

url = "https://baseapi.teld.cn/api/GetChargeListBySta/StaV3"

querystring = {"itemNumPerPage":"40","pageNum":"1","staID":"d4035b02-5f27-4ae9-b0b1-dd10010f5b7e"}


headers = {
    'cache-control': "no-cache",
    'postman-token': "928814f4-60c2-7c0c-a892-12409b8990bd"
    }

# response = requests.request("GET", url, headers=headers, params=querystring)
chargeList_detailedInfo = Get_py.get_chargelist_APP(stationId)
chargeList_detailedInfo

[{u'PileName': u'104\u53f7',
  u'carTypeName': u'\u672a\u77e5\u8f66\u578b',
  u'chargeTime': u'6\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNickName': u'\u81f3***\u751f'},
 {u'PileName': u'102\u53f7',
  u'carTypeName': u'\u745e\u9a70EC35',
  u'chargeTime': u'7\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNickName': u'1***2'},
 {u'PileName': u'105\u53f7',
  u'carTypeName': u'\u745e\u9a70EC35',
  u'chargeTime': u'8\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNickName': u'1***2'},
 {u'PileName': u'105\u53f7',
  u'carTypeName': u'\u745e\u9a70EC35',
  u'chargeTime': u'8\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNickName': u'1***2'},
 {u'PileName': u'104\u53f7',
  u'carTypeName': u'\u745e\u9a70EC35',
  u'chargeTime': u'8\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNickName': u'1***2'},
 {u'PileName': u'101\u53f7',
  u'carTypeName': u'\u745e\u9a70EC35',
  u'chargeTime': u'8\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNi

In [7]:
i = 0
fail = 0
hour_uni = u'\u5c0f\u65f6\u524d'
minute_uni = u'\u5206\u949f\u524d'
for stationID in station_list[100:]:
    stationID = stationID[0]
    try:
        chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID)
    except requests.exceptions.ConnectionError as e:
        try:
            chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID)
        except requests.exceptions.ConnectionError as e:
            fail += 1
            pass
    if len(chargeList_detailedInfo) == 0:
        i += 1
    else:
        for user_act in chargeList_detailedInfo:
            pileName = str(user_act['PileName'].encode('utf-8'))
            if user_act['userNickName'] is not None:
                userNickName = str(user_act['userNickName'].encode('utf-8'))
            else:
                userNickName = None
            if user_act['carTypeName'] is not None:
                carTypeName = str(user_act['carTypeName'].encode('utf-8'))
            else:
                carTypeName = None
            startTime = user_act['chargeTime']
            #AI detect startTime
            if len(startTime) < 6:
            #if in the expression of hours:
                if hour_uni in startTime:
                    idx_hour = len(startTime)-3
                    hours = int(startTime[:idx_hour])
                    startTime = startTime = datetime.datetime.now() - datetime.timedelta(hours=hours)
#                     startTime = '{:%Y-%m-%d %H:%M:%S}'.format(startTime)+"-07"
                    startTime = '{:%Y-%m-%d %H:%M}'.format(startTime)+"-07"
            #if in the expression of minutes:
                if minute_uni in startTime:
                    idx_min = len(startTime)-3
                    minutes = int(startTime[:idx_min])
                    startTime = startTime = datetime.datetime.now() - datetime.timedelta(minutes=minutes)
#                     startTime = '{:%Y-%m-%d %H:%M:%S}'.format(startTime)+"-07"
                    startTime = '{:%Y-%m-%d %H:%M}'.format(startTime)+"-07"
            elif len(startTime) >= 6:
                startTime = '2017-' + startTime + '-07'                    
            conn = Insert_py.insert_userActivity_info(conn, startTime, pileName, stationID, carTypeName, userNickName)
print(str(i) + " stations do not have activity history.")
print(str(fail) + " connections failed attempt.")

invalid input syntax for type timestamp with time zone: "刚刚"
LINE 4: ... (SELECT 1 FROM tbl_app_useractivity where startTime='刚刚')
                                                                ^

4230 stations do not have activity history.
3 connections failed attempt.


In [8]:
## ----- Test for single looping of multiprocessing post_request and insert -----##
import os
import psutil
pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('Before runnning script, memory use:', memoryUse, 'MB')
# %load_ext memory_profiler
# from memory_profiler import profile
# @profile 
# def test():
loop_num = 1
max_proc =900
time_out = 1000
for run in range(loop_num):
    tmp_pilllist = pillcode_list[run*max_proc:(run+1)*max_proc]
    t = time.time()
    if __name__ == '__main__':
        proc_record = []
        conn_list = []
        for i in range(len(tmp_pilllist)):
            conn = indb.init_db(db_name, user_name, password, host, port)
            conn_list.append(conn)
            p = Process(target=pillState_scrap, args=(conn, hdr, tmp_pilllist[i][0], time_out))
            p.start()
            proc_record.append(p)
        for p in proc_record:
            p.join(1)
        for conn in conn_list:
            conn.close()
        for p in proc_record:
            p.terminate()
#         del proc_record
#         del conn_list
        print("success, time costs is " + str(time.time()-t))   
pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('After runnning script,memory use:', memoryUse, 'MB')

('Before runnning script, memory use:', 72.70703125, 'MB')
2.36052203178
2.09582901001
1.90273499489
2.14482903481
2.27234387398
2.26481103897
2.84621310234
3.16032314301
3.12021493912
3.50480318069
3.43114900589
3.39744997025
3.54908490181
3.18675088882
2.90019416809
3.76884484291
3.79637789726
3.85017991066
3.7966940403
3.93263792992
3.83353590965
3.75909304619
3.99796581268
4.29836082458
4.17587399483
3.96918678284
4.66725301743
4.20207691193
4.86533904076
4.51942491531
4.649559021
4.36551213264
4.33054184914
4.78237104416
4.40960407257
4.40417098999
4.18951797485
4.62285804749
4.50949788094
5.30450892448
3.66499495506
5.42893004417
4.41705012321
3.56230211258
3.86556506157
3.70444393158
4.36553812027
4.07809901237
4.85262703896
4.37795305252
4.27859592438
4.15128803253
4.35490202904
4.67027688026
4.46108198166
3.99541497231
4.44550299644
3.55627822876
4.87362790108
5.08655595779
3.70096707344
4.8728659153
5.0523879528
3.95606207848
3.04310107231
3.27356219292
3.54867696762
2.790940

In [13]:
%memit test

ERROR:root:Line magic function `%memit` not found.


In [15]:
def main():
    """The main process"""
    global displaytime
    global endTime 
    global max_proc
    global loop_no 
    global rest_no
    global time_out
    proxies_pool = None
    t = time.time()
    while True:
        if datetime.datetime.now() >= endTime:
            break
        for loopNo in range(loop_no):
            tmp_pilllist = pillcode_list[loopNo*max_proc:(loopNo+1)*max_proc]
            proc_record = []
            conn_list = []                
            for i in range(len(tmp_pilllist)):
                conn = indb.init_db(db_name, user_name, password, host, port)
                conn_list.append(conn)
                p = Process(target=pillState_scrap, args=(conn, hdr, tmp_pilllist[i][0], time_out,))
                p.start()
                proc_record.append(p)
            for p in proc_record:
                p.join()
            for conn in conn_list:
                conn.close()
            for p in proc_record:
                p.terminate()
#             del proc_record
#             del conn_list
            if datetime.datetime.now() >= displaytime:
                print("successfully looped {} times".format(loopNo+1))
                displaytime += datetime.timedelta(hours=1)
        tmp_pilllist = pillcode_list[loop_no*max_proc:]
        proc_record = []
        conn_list = [] 
        for i in range(len(tmp_pilllist)):
            conn = indb.init_db(db_name, user_name, password, host, port)
            conn_list.append(conn)
            p = Process(target=pillState_scrap, args=(conn, hdr, tmp_pilllist[i][0], time_out,))
            p.start()
            proc_record.append(p)
        for p in proc_record:
            p.join()
        for conn in conn_list:
            conn.close()
        for p in proc_record:
            p.terminate()
#         del proc_record
#         del conn_list
        print("time for a sleep zz..")
        print("One process runs for "+str(time.time()-t))
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0]/2.**20 #in MB
        print('After runnning script,memory use:', memoryUse, 'MB')
        time.sleep(900)

if __name__ == '__main__':
    import os
    import psutil
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0]/2.**20 #in MB
    print('Before runnning script, memory use:', memoryUse, 'MB')
    
    displaytime = datetime.datetime.now() + datetime.timedelta(hours=1)
    endTime = datetime.datetime.now() + datetime.timedelta(hours=1)
    time_out = 1000
    max_proc = 900
    loop_no = len(pillcode_list)//max_proc
    rest_no = len(pillcode_list)%max_proc
    main()

('Before runnning script, memory use:', 109.6640625, 'MB')
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
successfully looped 37 times
successfully looped 74 times
time for a sleep zz..
One process runs for 7792.00441408
('After runnning script,memory use:', 143.51953125, 'MB')


In [9]:
# pilecode = '1101060314110'
# pile_detailedInfo = Get_py.get_onePile_APP(hdr, pilecode)
# conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)
# REQUEST_URL = 'http://baseapi.teld.cn/api/GetPillDetails/StaV2?pillCode='+pilecode
# pile_resp = requests.get(REQUEST_URL, headers=hdr)


In [10]:
# json_requestData = pile_resp.json()
# json_requestData
# pile_detailedInfo = json_requestData['data']['terminal']

In [11]:
# stationId = '044a0e7c-419e-477e-aafb-8d4b9eea1475'
# piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)
# pile_detailedInfo = Get_py.get_onePile_APP(hdr, piles_info[3]['code'])

In [12]:
# pile_detailedInfo
# conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)

In [13]:
##------- Try fetching all the data through psycopg2 -------## 
# try:
#     cur = conn.cursor()
#     cur.execute("select stationid from tbl_app_eachstations limit 10;")
#     exists = cur.fetchall()
#     cur.close()
# except psycopg2.Error as e:
#     print(e)
# for i in range(len(exists)):
#     print(exists[i][0])

In [14]:
# REQUEST_URL = 'https://www.teld.cn/StationNetwork/GetStationNetword?ProvinceName=&CityName=&KeyWords=&RegionName=& \
#                             type=&page=1&rows=10'
# # req = urllib.request.Request(REQUEST_URL) 
# hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/ \
#                        59.0.3071.115 Safari/537.36', 
#        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
# response = urllib2.Request(REQUEST_URL, headers=hdr)

# try:
#     page = urllib2.urlopen(response)
# except urllib2.HTTPError, e:
#     print e.fp.read()
    
# content = json.load(page)
# print(content['rows'][0])
# response_data = urllib.request.urlopen(REQUEST_URL)
# d = json.load(response_data)
# print('number of results %d' % (len(d)))
# type(response_data)